# Fixed Point Analysis
This notebook is part of the [SachsLab Workshop for Intracranial Neurophysiology and Deep Learning](https://github.com/SachsLab/IntracranialNeurophysDL).

Follow the link below to run in Google Colab or continue if running locally.

Run the first few cells to normalize Local / Colab environments, then proceed below for the lesson.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/SachsLab/IntracranialNeurophysDL/blob/master/notebooks/07_02_fixed_point_analysis.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/SachsLab/IntracranialNeurophysDL/blob/master/notebooks/07_02_fixed_point_analysis.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


In [1]:
import os
import sys
from pathlib import Path
try:
    # See if we are running on google.colab
    import google.colab
    from google.colab import files
    
    os.chdir('..')
    if not (Path.home() / '.kaggle').is_dir():
        # Configure kaggle
        files.upload()  # Find the kaggle.json file in your ~/.kaggle directory.
        !pip install -q kaggle
        !mkdir -p ~/.kaggle
        !mv kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
        
    if sys.version_info > (3, 0):
        from importlib import reload
        !pip install --upgrade -q https://storage.googleapis.com/jax-wheels/cuda$(echo $CUDA_VERSION | sed -e 's/\.//' -e 's/\..*//')/jaxlib-0.1.28-cp36-none-linux_x86_64.whl
    else:
        !pip install --upgrade -q https://storage.googleapis.com/jax-wheels/cuda$(echo $CUDA_VERSION | sed -e 's/\.//' -e 's/\..*//')/jaxlib-0.1.28-cp27-none-linux_x86_64.whl
    
    !pip install --upgrade -q git+https://github.com/google/jax.git
    
    if Path.cwd().stem != 'IntracranialNeurophysDL':
        if not (Path.cwd() / 'IntracranialNeurophysDL').is_dir():
            # Download the workshop repo and change to its directory
            !git clone --recursive https://github.com/SachsLab/IntracranialNeurophysDL.git
        os.chdir('IntracranialNeurophysDL')
    IN_COLAB = True
except ModuleNotFoundError:
    IN_COLAB = False
    if Path.cwd().stem == 'notebooks':
        os.chdir(Path.cwd().parent)
    # Make sure the kaggle executable is on the PATH
    os.environ['PATH'] = os.environ['PATH'] + ';' + str(Path(sys.executable).parent / 'Scripts')

Saving kaggle.json to kaggle.json
     |████████████████████████████████| 47.5MB 1.1MB/s 
Cloning into 'IntracranialNeurophysDL'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 1180 (delta 26), reused 19 (delta 8), pack-reused 1127
Receiving objects: 100% (1180/1180), 311.80 MiB | 36.67 MiB/s, done.
Resolving deltas: 100% (703/703), done.
Checking out files: 100% (93/93), done.


## Get code from google research repository

In [0]:
if not (Path.cwd() / '..' / 'ctd').is_dir():
    !git clone --recursive https://github.com/google-research/computation-thru-dynamics.git ../ctd
    !mv ../ctd/fixed_point_finder ./
import fixed_point_finder.decision as decision
import fixed_point_finder.fixed_points as fp_optimize
import fixed_point_finder.rnn as rnn
import fixed_point_finder.utils as utils

In [0]:
import jax.numpy as np
from jax import random
from jax.experimental import optimizers
#config.update("jax_debug_nans", True) # Useful for finding numerical errors
import matplotlib.pyplot as plt
import numpy as onp  # original CPU-backed NumPy
import time

plt.rcParams.update({'font.size': 22})
%load_ext autoreload
%autoreload 2

## Load the trained LFADS model
See the 06_01 notebook.


In [0]:
datadir = Path.cwd() / 'data' / 'joeyo'
network_path = str(datadir / 'lfads_model.npz')
loaded_params = onp.load(network_path)
trained_params = loaded_params['arr_0'].item()

## Notes

1. Use some known initial states to evolve some RNN internal states.
2. Use RNN internal states as initial fixed point candidates.
3. Optimize to reduce the loss between internal state and internal state after step.
    * Note: `opt_init` is called with the candidate fixed points as 'parameters'.
4. Tolerance cutoff, unique, exclude outliers, sort by loss.
5. Plot "best" fixed points in PC space.